# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [56]:
path = '../03-corpus/1-crawler/'
acteur = "chum"
#pdfs = False

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [57]:
from pandas import *

regex = '.*png.*|.*jpeg.*|.*jpg.*|.*docx.*|.*js.*|.*font.*|.*gif.*|.*formulaire.*|.*?f%5B0%5D.*|.*img.*|.*%5Bfilter%.*|.*css.*|.*scripts.*|.*zip.*|.*xlsx.*|.*cms.*|.*/images/.*|.*sondage.*|.*/depenses/.*|.*demandes-acces.*'
# if not pdfs:
#     regex += '|.*pdf.*'

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur + '.csv', encoding='UTF-8') as f:
    csv = read_csv(f, sep=';')
    # Nettoyer ce qui ne devrait pas se trouver là
    csv = csv[~csv["Address"].str.contains(regex, case=False)] 
csv

,Address,Type,Title,Charset,Description,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,https://www.chumontreal.qc.ca/,text/html,Accueil - CHUM,utf-8,Bienvenue sur le site web du Centre hospitalie...,NaN,NaN,NaN
1,https://www.chumontreal.qc.ca/accueil,text/html,Centre hospitalier de l’Université de Montréal,utf-8,NaN,NaN,NaN,NaN
2,https://www.chumontreal.qc.ca/user/login,text/html,Se connecter,utf-8,NaN,NaN,NaN,NaN
3,https://www.chumontreal.qc.ca/patients,text/html,Patients - CHUM,utf-8,Cette section regroupe les plus récentes infor...,NaN,NaN,NaN
4,https://www.chumontreal.qc.ca/enseignement-aca...,text/html,Enseignement et Académie CHUM,utf-8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4193,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html,Tout savoir sur le traitement par plaque radio...,utf-8,NaN,NaN,NaN,NaN
4194,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html,Qu’est-ce que la stéréotaxie et le CyberKnife?,utf-8,NaN,NaN,NaN,NaN
4195,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html,Comment se passe l’énucléation de l’oeil suiva...,utf-8,NaN,NaN,NaN,NaN
4196,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html,Une équipe à l’affût des innovations en format...,utf-8,NaN,NaN,NaN,NaN


In [58]:
fr = csv[~csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en français
en = csv[csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en anglais
print("On va tenter d'aspirer {} pages Web".format(len(fr) + len(en)))

fr

On va tenter d'aspirer 4177 pages Web


,Address,Type
0,https://www.chumontreal.qc.ca/,text/html
1,https://www.chumontreal.qc.ca/accueil,text/html
2,https://www.chumontreal.qc.ca/user/login,text/html
3,https://www.chumontreal.qc.ca/patients,text/html
4,https://www.chumontreal.qc.ca/enseignement-aca...,text/html
...,...,...
4193,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html
4194,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html
4195,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html
4196,https://www.chumontreal.qc.ca/patients/cicc/ca...,text/html


In [59]:
en

,Address,Type
38,https://www.chumontreal.qc.ca/en/user/login,text/html
73,https://www.chumontreal.qc.ca/en/nouvelles,text/html
203,https://www.chumontreal.qc.ca/en/fiches-sante,text/html
249,https://www.chumontreal.qc.ca/en/nouvelles/frq...,text/html
251,https://www.chumontreal.qc.ca/en/taxonomy/term/1,text/html
...,...,...
3887,https://www.chumontreal.qc.ca/en/nouvelles?fie...,text/html
4063,https://www.chumontreal.qc.ca/en/fiches-sante?...,text/html
4078,https://www.chumontreal.qc.ca/en/fiches-sante?...,text/html
4091,https://www.chumontreal.qc.ca/en/fiches-sante?...,text/html


In [60]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextHTML(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav', 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]',
    'div[class="container-fluid container-blue container-dl-menu"]', 'div[class="col-xs-12 dl-menuwrapper menu-mobile visible-x"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()


    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data

In [61]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextHTML(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav', 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]',
    'div[class="container-fluid container-blue container-dl-menu"]', 'div[class="col-xs-12 dl-menuwrapper menu-mobile visible-x"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()

    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data

In [62]:
import io
from PyPDF2 import PdfReader

def getTextPDF(url):
    pdf_link = requests.get(url)
    with io.BytesIO(pdf_link.content) as f:
        reader = PdfReader(f)
        nb_pages = len(reader.pages)
        text = ''
        if nb_pages <= 15: # On ne prendra pas les documents de plus de 15 pages
            for i in range(nb_pages):
                page = reader.pages[i]
                text += page.extract_text().lower().replace('\n', '')
        if text :
            return text


In [63]:
def scrape_list(df):
    output = []
    pdfs = df[df['Type'] == 'application/pdf']['Address'].tolist()
    htmls = df[df['Type'] == 'text/html']['Address'].tolist() 
    # Il y a aussi des pages spour lesquelles le crawler n'a pas été en mesure de consigner une valeur
    # On va essayer d'aller chercher leur contenu avec la fonction getTextHTML 
    nans = df[df["Type"].isnull()]['Address'].tolist()  
    htmls = htmls + nans

    for url in htmls :
        try: 
            text = getTextHTML(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'text/html', 'text':text})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    for url in pdfs :
        try: 
            text = getTextPDF(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'application/pdf', 'text':text})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    return output

In [64]:
sites_fr = scrape_list(fr)
if(len(en) > 0):
    sites_en = scrape_list(en)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.chumontreal.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.chumontreal.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.chumontreal.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e

ERROR  - https://www.chumontreal.qc.ca/sites/default/files/2021-07/carte_nos_espaces_juin_2021_web.pdf
HTTPSConnectionPool(host='www.chumontreal.qc.ca', port=443): Max retries exceeded with url: /sites/default/files/2021-07/carte_nos_espaces_juin_2021_web.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))
ERROR  - https://www.chumontreal.qc.ca/sites/default/files/inline-files/201009_guide-patient-2020.pdf
HTTPSConnectionPool(host='www.chumontreal.qc.ca', port=443): Max retries exceeded with url: /sites/default/files/inline-files/201009_guide-patient-2020.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))
ERROR  - https://www.chumontreal.qc.ca/sites/default/files/2021-07/carte_nos_espaces_juin_2021_web_1.pdf
HTTPSConnectionPool(host='www.chumontreal.qc.ca

In [ ]:
output_path = '../03-corpus/2-data/'

sites_fr = pd.DataFrame(sites_fr)
sites_fr = csv.merge(sites_fr, how='right', on='Address')
sites_fr = sites_fr[['Address', 'Title', 'Type', 'text']]

sites_fr

,Address,Title,Type,text
0,https://www.ramq.gouv.qc.ca/fr,Accueil | Régie de l’assurance maladie du Québ...,text/html,Avez-vous déménagé? Avez-vous fait votre chang...
1,https://www.ramq.gouv.qc.ca/en,Home | Régie de l’assurance maladie du Québec ...,text/html,Have you moved? Have you made your change of a...
2,https://www.ramq.gouv.qc.ca/fr/nous-joindre,Nous joindre | Régie de l’assurance maladie du...,text/html,Nous joindre Services couverts hors Québec : t...
3,https://www.ramq.gouv.qc.ca/fr/citoyens/assura...,Assurance maladie | Régie de l’assurance malad...,text/html,Assurance maladie La RAMQ administre le régime...
4,https://www.ramq.gouv.qc.ca/fr/citoyens/assura...,Assurance médicaments | Régie de l’assurance m...,text/html,Assurance médicaments Toute personne établie a...
...,...,...,...,...
3450,https://www.ramq.gouv.qc.ca/sites/default/file...,"<div class=""file""><span class=""file_name"">Télé...",application/pdf,adresse du représentant représentant légal : ...
3451,https://www.ramq.gouv.qc.ca/sites/default/file...,"<div class=""file""><span class=""file_name"">Télé...",application/pdf,demande de renseignements concernant une perso...
3452,https://www.ramq.gouv.qc.ca/sites/default/file...,"<div class=""file""><span class=""file_name"">Télé...",application/pdf,sécuritésociale protocole d’entente entre le g...
3453,https://www.ramq.gouv.qc.ca/sites/default/file...,"<div class=""file""><span class=""file_name"">Télé...",application/pdf,service de la contribution et de l’aide fina...


In [ ]:
sites_fr.to_csv(output_path + '1-fr/' + acteur + '.csv', escapechar='/')
if(len(en) > 0):
    sites_en = pd.DataFrame(sites_en)
    sites_en = csv.merge(sites_en, how='right', on='Address')
    sites_en = sites_en[['Address', 'Title', 'Type', 'text']]
    sites_en.to_csv(output_path + '1-en/' + acteur + '_en.csv', escapechar='/')